In [ ]:
import numpy as np          # 数値計算ライブラリ
import matplotlib.pyplot as plt    # グラフ描画ライブラリ
import torch                 # 機械学習ライブラリPyTorch
from torch import nn        # PyTorchのニューラルネットワークモジュール
from torchviz import make_dot      # PyTorchのグラフ可視化ツール
import japanize_matplotlib  # matplotlibの日本語表示を可能にするライブラリ
import torch.optim as optim # PyTorchの最適化アルゴリズムを定義するoptimモジュール
import pandas as pd         # データフレーム処理のためのライブラリ
import sklearn              # 機械学習ライブラリscikit-learn
from torchinfo import summary   # モデルのサマリー情報を表示するためのライブラリ
from sklearn.model_selection import train_test_split  # データのトレーニングとテストの分割を行う関数
from sklearn.metrics import accuracy_score   # 正解率を計算するための関数


# 以下可視性のために定義。
import inspect # フレームを取得するためのライブラリ
from IPython.display import display # データフレームを表示するためのライブラリ

# 引数の変数名を出力する変数。ただし仕様上、関数を呼び出した場所と同スコープの変数なら1。その一個上なら2，さらにひとつ上なら3にしなければならない。
def print_var_name(var,n=1):
    # 現在のフレームを取得する
    current_frame = inspect.currentframe()
    # 現在のフレームのn回外側のフレームを取得する
    outer_frame = current_frame
    for _ in range(n):
        outer_frame =outer_frame.f_back
    # 外側のフレームのローカル変数を取得する。2次元タプル?がずらっと配列で並んでいる。
    locals_dict = outer_frame.f_locals
    # 変数名を取得する
    var_name = [k for k, v in locals_dict.items() if v is var]
    if not var_name:
        print("変数が見つかりませんでした。")
    else:
        # 変数名を出力する
        print("変数名 : ",var_name[0]," 変数型は ",type(var))

def dataframe_converter(func):
    def wrapper(*args, **kwargs):
        for arg in args:
            
            try:
                print("形状は",arg.shape)
            except:
                pass

            try:
                # 引数がNumPy配列の場合はPandasデータフレームに変換する
                print_var_name(arg,2)
                if isinstance(arg, np.ndarray):
                    df = pd.DataFrame(arg)
                    # デコレータで修飾された関数にデータフレームを渡す
                    func(df)
                elif isinstance(arg, torch.Tensor):
                    df = pd.DataFrame(arg.detach().numpy())
                    # デコレータで修飾された関数にデータフレームを渡す
                    func(df)
                elif isinstance(arg, sklearn.utils.Bunch):
                    df = pd.DataFrame(arg.data, columns=arg.feature_names)
                    # デコレータで修飾された関数にデータフレームを渡す
                    func(df)
                else:
                    # デコレータで修飾された関数にそのままのオブジェクトを渡す
                    func(arg)
            except:
                # デコレータで修飾された関数にそのままのオブジェクトを渡す
                func(arg)
    return wrapper

@dataframe_converter
def display_custom(obj,head=True): # 引数のオブジェクトを表示する。可視性を高めるためにpdに変更するがおそらく大規模の場合は乱用はパフォーマンスに影響する。
    # 引数のオブジェクトを表示する
    if head and isinstance(obj, pd.DataFrame):
        display(obj.head())
        
    else:
        display(obj)

In [ ]:
# デバイスの設定
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("使用デバイス：", torch.cuda.is_available())



In [ ]:
# MNIStデータセットのダウンロード
import torchvision.datasets as datasets

# ダウンロード先のディレクトリを指定
data_root = "./mnist_data"

train_set0 = datasets.MNIST(root=data_root, train=True, download=True)



In [ ]:
!nvidia-smi

In [ ]:
print(type(train_set0))
print(len(train_set0)) #__len__メソッドの出力
image,label = train_set0[0] #__getitem__メソッドの出力を変数に格納
print('入力データの型 : ',type(image))
print('正解データの型 : ',type(label))
# 画像のサイズとモードを出力
print(image.size)  # (640, 480)
print(image.mode)  # RGB

# (x, y) = (10,10)のピクセルの値を出力
pixel_value = image.getpixel((10, 10))
print(pixel_value)  

# すべてのピクセルの明度値を取得
pixel_values = list(image.getdata())

# 明度値を表示
print(pixel_values)



In [ ]:
# train_set0の属性を出力
print(train_set0.train_data.shape)   # torch.Size([60000, 28, 28])
print(train_set0.train_labels.shape) # torch.Size([60000])
print(train_set0.classes)        # ['0 - zero', '1 - one', '2 - two', '3 - three', '4 - four', '5 - five', '6 - six', '7 - seven', '8 - eight', '9 - nine']

In [ ]:
# 入力データの画像の表示
plt.figure(figsize=(10,10))
plt.title(f'{label}')
plt.imshow(image, cmap='gray_r')
plt.axis('off')
plt.show()

In [ ]:
# 正解データ付きて、最初の20個の画像を表示
plt.figure(figsize=(10,3))
for i in range(20):
    ax=plt.subplot(2,10,i+1)

    # imageとlabelを取得
    image,label = train_set0[i]

    # イメージを表示
    plt.imshow(image, cmap='gray_r')
    ax.set_title(f'{label}')
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

In [ ]:
import torchvision.transforms as transforms

# データの前処理を定義
transform1 = transforms.Compose([transforms.ToTensor()])

train_set1 = datasets.MNIST(root=data_root, train=True, download=True, transform=transform1)

# 変換結果の確認
image,label = train_set1[0]
print('変数の型 :',type(train_set1))
print('入力データの型 : ',type(image))
print('入力データのshape : ',image.shape)
print('最小値 : ',image.min())
print('最大値 : ',image.max())
print('データ数 : ',len(train_set1))
print(image[:,10:15,10:15])

# 変換前と比較
image, _ = train_set0[0] 
print('変換前:',list(image.crop((10,10,15,15)).getdata()))
lst = [x / 255 for x in list(image.crop((10,10,15,15)).getdata())]
lst_formatted = ['{:.4f}'.format(x) for x in lst]
print("正規化 :",lst_formatted)

In [ ]:
# -1~1で正規化をする
transform2 = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_set2 = datasets.MNIST(root=data_root, train=True, download=True, transform=transform2)

# 変換結果の確認
image,label = train_set2[0]
print('変数の型 :',type(train_set2))
print('入力データの型 : ',type(image))
print('入力データのshape : ',image.shape)
print('最小値 : ',image.min())
print('最大値 : ',image.max())
print('データ数 : ',len(train_set2))

In [ ]:
# 入力できるように1次元に変換
transform3 = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,)), transforms.Lambda(lambda x: x.view(-1))])

train_set3 = datasets.MNIST(root=data_root, train=True, download=True, transform=transform3)

# 変換結果の確認
image,label = train_set3[0]
print('変数の型 :',type(train_set3))
print('入力データの型 : ',type(image))
print('入力データのshape : ',image.shape)
print('最小値 : ',image.min())
print('最大値 : ',image.max())
print('データ数 : ',len(train_set3))
print(train_set3.train_data.shape)

In [ ]:
# 以上の例より改めて最終的な前処理を定義
# データ変換用関数Transform
# (1) ToTensor() : PIL形式の画像をPyTorchのテンソルに変換
# (2) Normalize() : [0,1]の範囲の画像を[-1,1]の範囲に正規化
# (3) Lambda() : 1つ1つのデータを[1,28,28]から[784]に変換
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,)), transforms.Lambda(lambda x: x.view(-1))])


In [ ]:
# 最終的なデータセットを作成

# 訓練データセットを定義
train_set = datasets.MNIST(root=data_root, train=True, download=True, transform=transform)

# 検証データセットを定義
test_set = datasets.MNIST(root=data_root, train=False, download=True, transform=transform)

In [ ]:
print(type(test_set))
print(len(test_set)) #__len__メソッドの出力
image,label = test_set[0] #__getitem__メソッドの出力を変数に格納
print('入力データの型 : ',type(image))
print('正解データの型 : ',type(label))
print('入力データのshape : ',image.shape)


In [ ]:
from torch.utils.data import DataLoader

# ミニバッチサイズ
batch_size = 500

# 訓練用データローダーを作成
# 訓練用なのでshuffle=True
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)

# 検証用データローダーを作成
# 訓練用ではないのでshuffle=False
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

print(type(train_loader))

# 何組のミニバッチがあるか
print(len(train_loader))
print(len(train_set)/500) #上記と同じ

# データローダーから最初のミニバッチを取り出す
images, labels = next(iter(train_loader))

# ミニバッチの中身を確認
print(type(images))
print(type(labels))
print(images.shape)
print(labels.shape)



In [ ]:
# 入力次元数
n_input = image.shape[0]

# 出力次元数=分類先のクラス数
n_output = len(train_set0.classes)

# 中間層の次元数
n_hidden = 128


In [ ]:
# モデルの定義
# 784 -> 128 -> 10のにニューラルネットワーク

class Net(nn.Module):
    def __init__(self,n_input,n_hidden,n_output):

        super().__init__()
        
        #隠れ層の定義
        self.l1 = nn.Linear(n_input,n_hidden)

        #出力層の定義
        self.l2 = nn.Linear(n_hidden,n_output)

        # ReLU関数の定義
        self.relu = nn.ReLU(inplace=True)

    def forward(self,x):
        x = self.l1(x)
        x = self.relu(x)
        x = self.l2(x)
        return x

In [ ]:
# 乱数の固定
torch.manual_seed(123)
torch.cuda.manual_seed(123)

# モデル変数の生成
net = Net(n_input,n_hidden,n_output)

# モデルをGPUに転送
net = net.to(device)

In [ ]:
# 損失関数の定義
criterion = nn.CrossEntropyLoss()

# 最適化手法の定義
optimizer = optim.SGD(net.parameters(), lr=0.01)

# 学習回数
n_epoch = 100

# 評価結果を格納するリスト(エポック数、訓練データの損失、テストデータの損失、訓練データの正解率、テストデータの正解率)
history = np.zeros((0,5))

# モデルのパラメータの確認
for parameter in net.named_parameters():
    print(parameter)

In [ ]:
print(net)

In [21]:
# tqdmを使って進捗状況を表示
from tqdm import tqdm


# 繰り返し処理

for epoch in tqdm(range(n_epoch)):

    train_acc, train_loss = 0, 0
    test_acc, test_loss = 0, 0
    n_train, n_test = 0, 0

    # 訓練フェーズ
    for inputs,labels in tqdm(train_loader):
        n_train += len(inputs)
        # GPUに転送
        inputs, labels = inputs.to(device), labels.to(device)

        # 勾配の初期化
        optimizer.zero_grad()

        # 順伝播
        outputs = net(inputs)

        # 損失の計算
        loss = criterion(outputs, labels)

        # 逆伝播
        loss.backward()

        # パラメータの更新
        optimizer.step()
        # 正解率の計算
        _, predicted_train = torch.max(outputs.data, 1) #出力の確率分布から最大値があるインデックスを取得しpredicted_trainに格納。つまり予測ラベルとなる。
        train_acc += (predicted_train == labels).sum().item()

    # 予測フェーズ
    for inputs_test, labels_test in test_loader:
        n_test += len(inputs_test)
        # GPUに転送
        inputs_test, labels_test = inputs_test.to(device), labels_test.to(device)

        # 順伝播
        outputs_test = net(inputs_test)

        # 損失の計算
        loss_test = criterion(outputs_test, labels_test)

        # 正解率の計算
        _, predicted = torch.max(outputs_test.data, 1)
        test_acc += (predicted == labels_test).sum().item()

    # 記録用リストに記録
    if (epoch)%100 == 0:
        history = np.vstack((history, np.array([[epoch, loss.item(), loss_test.item(), train_acc, test_acc]])))


        

  0%|          | 14/10000 [02:15<26:56:30,  9.71s/it]


KeyboardInterrupt: 